In [1]:
import psycopg
from psycopg import sql

In [2]:
HOST     = 'localhost'
DBNAME   = 'mydbase'
SCNAME   = 'myschema'
USER     = 'myadmin'
PASSWORD = 'qwer1234'

In [3]:
CONNECTION = f"host={HOST} dbname={DBNAME} user={USER} password={PASSWORD}"

# Drop tables

In [4]:
with psycopg.connect(CONNECTION) as conn:

    conn.execute(f'DROP TABLE IF EXISTS {SCNAME}."course";')
    conn.execute(f'DROP TABLE IF EXISTS {SCNAME}."instructor";')
    conn.execute(f'DROP TABLE IF EXISTS {SCNAME}."department";')

    conn.commit()

# Create tables

In [5]:
with psycopg.connect(CONNECTION) as conn:

    conn.execute(
        sql.SQL("""
            CREATE TABLE IF NOT EXISTS {table}(
                "dept_name" VARCHAR(20),
                "building" VARCHAR(15),
                "budget" NUMERIC(12,2) CHECK ("budget" > 0),
                PRIMARY KEY ("dept_name")
            )
        """).format(table = sql.Identifier(SCNAME, 'department'))
    )

    conn.commit()

In [6]:
with psycopg.connect(CONNECTION) as conn:

    conn.execute(
        sql.SQL("""
            CREATE TABLE IF NOT EXISTS {table}(
                "course_id" VARCHAR(8),
                "title" VARCHAR(50),
                "dept_name" VARCHAR(20),
                "credits" NUMERIC(2,0) CHECK ("credits" > 0),
                PRIMARY KEY ("course_id"),
                FOREIGN KEY ("dept_name") REFERENCES {ref_table}({ref_col})
                    ON DELETE SET NULL
            )
        """).format(table = sql.Identifier(SCNAME, 'course'),
                    ref_table = sql.Identifier(SCNAME, 'department'),
                    ref_col   = sql.Identifier('dept_name')
    ))

    conn.commit()

In [7]:
with psycopg.connect(CONNECTION) as conn:

    conn.execute(
        sql.SQL("""
            CREATE TABLE IF NOT EXISTS {table}(
                {col1} VARCHAR(5),
                {col2} VARCHAR(20) NOT NULL,
                {col3} VARCHAR(20),
                {col4} NUMERIC(8,2) CHECK ({col4} > 29000),
                PRIMARY KEY ({col1}),
                FOREIGN KEY ({col3}) REFERENCES {ref_table}({ref_col})
                    ON DELETE SET NULL
            )
        """).format(table = sql.Identifier(SCNAME, 'instructor'),
                    col1  = sql.Identifier('id'),
                    col2  = sql.Identifier('name'),
                    col3  = sql.Identifier('dept_name'),
                    col4  = sql.Identifier('salary'),
                    ref_table = sql.Identifier(SCNAME, 'department'),
                    ref_col   = sql.Identifier('dept_name')
    ))

    conn.commit()

In [8]:
with psycopg.connect(CONNECTION) as conn:

    cur = conn.execute("""
        SELECT "table_name"
        FROM "information_schema"."tables"
        WHERE "table_schema" = %s
        ORDER BY "table_name";
    """, (SCNAME,))

    records = cur.fetchall()
    for r in records:
        print(r)

('course',)
('department',)
('instructor',)


# Insert records (department)

In [9]:
with psycopg.connect(CONNECTION) as conn:

    ## clear
    conn.execute(sql.SQL("DELETE FROM {}").format(sql.Identifier(SCNAME, 'department')))

    ## prepared statement
    query = sql.SQL("INSERT INTO {} VALUES (%s, %s, %s)").format(sql.Identifier(SCNAME, 'department'))
    
    with conn.pipeline() as p:
        conn.execute(query, ('Biology', 'Watson', '90000'))
        conn.execute(query, ('Comp. Sci.', 'Taylor', '100000'))
        conn.execute(query, ('Elec. Eng.', 'Taylor', '85000'))
        conn.execute(query, ('Finance', 'Painter', '120000'))
        conn.execute(query, ('History', 'Painter', '50000'))
        conn.execute(query, ('Music', 'Packard', '80000'))
        conn.execute(query, ('Physics', 'Watson', '70000'))

    conn.commit()    

In [10]:
with psycopg.connect(CONNECTION) as conn:
    
    cur = conn.execute(sql.SQL("SELECT * FROM {}").format(sql.Identifier(SCNAME, 'department')))

    records = cur.fetchall()
    for r in records:
        print(r)

('Biology', 'Watson', Decimal('90000.00'))
('Comp. Sci.', 'Taylor', Decimal('100000.00'))
('Elec. Eng.', 'Taylor', Decimal('85000.00'))
('Finance', 'Painter', Decimal('120000.00'))
('History', 'Painter', Decimal('50000.00'))
('Music', 'Packard', Decimal('80000.00'))
('Physics', 'Watson', Decimal('70000.00'))


# Insert records (course)

In [11]:
with psycopg.connect(CONNECTION) as conn:

    ## clear
    conn.execute(sql.SQL("DELETE FROM {}").format(sql.Identifier(SCNAME, 'course')))

    ## prepared statement
    query = sql.SQL("INSERT INTO {} VALUES ({})").format(
        sql.Identifier(SCNAME, 'course'),
        sql.SQL(', ').join(sql.Placeholder() * 4)
    )
    
    with conn.pipeline() as p:
        conn.execute(query, ('BIO-101', 'Intro. to Biology', 'Biology', '4'))
        conn.execute(query, ('BIO-301', 'Genetics', 'Biology', '4'))
        conn.execute(query, ('BIO-399', 'Computational Biology', 'Biology', '3'))
        conn.execute(query, ('CS-101', 'Intro. to Computer Science', 'Comp. Sci.', '4'))
        conn.execute(query, ('CS-190', 'Game Design', 'Comp. Sci.', '4'))
        conn.execute(query, ('CS-315', 'Robotics', 'Comp. Sci.', '3'))
        conn.execute(query, ('CS-319', 'Image Processing', 'Comp. Sci.', '3'))
        conn.execute(query, ('CS-347', 'Database System Concepts', 'Comp. Sci.', '3'))
        conn.execute(query, ('EE-181', 'Intro. to Digital Systems', 'Elec. Eng.', '3'))
        conn.execute(query, ('FIN-201', 'Investment Banking', 'Finance', '3'))
        conn.execute(query, ('HIS-351', 'World History', 'History', '3'))
        conn.execute(query, ('MU-199', 'Music Video Production', 'Music', '3'))
        conn.execute(query, ('PHY-101', 'Physical Principles', 'Physics', '4'))

    conn.commit()    

In [12]:
with psycopg.connect(CONNECTION) as conn:
    
    cur = conn.execute(sql.SQL("SELECT * FROM {}").format(sql.Identifier(SCNAME, 'course')))

    records = cur.fetchall()
    for r in records:
        print(r)

('BIO-101', 'Intro. to Biology', 'Biology', Decimal('4'))
('BIO-301', 'Genetics', 'Biology', Decimal('4'))
('BIO-399', 'Computational Biology', 'Biology', Decimal('3'))
('CS-101', 'Intro. to Computer Science', 'Comp. Sci.', Decimal('4'))
('CS-190', 'Game Design', 'Comp. Sci.', Decimal('4'))
('CS-315', 'Robotics', 'Comp. Sci.', Decimal('3'))
('CS-319', 'Image Processing', 'Comp. Sci.', Decimal('3'))
('CS-347', 'Database System Concepts', 'Comp. Sci.', Decimal('3'))
('EE-181', 'Intro. to Digital Systems', 'Elec. Eng.', Decimal('3'))
('FIN-201', 'Investment Banking', 'Finance', Decimal('3'))
('HIS-351', 'World History', 'History', Decimal('3'))
('MU-199', 'Music Video Production', 'Music', Decimal('3'))
('PHY-101', 'Physical Principles', 'Physics', Decimal('4'))


# Insert records (instructor)

In [13]:
with psycopg.connect(CONNECTION) as conn:

    ## clear
    conn.execute(sql.SQL("DELETE FROM {}").format(sql.Identifier(SCNAME, 'instructor')))

    ## template
    SQL_TEMPLATE_INSERT = "INSERT INTO {0} ({1}) VALUES ({2})"

    ## columns
    COLNAMES_INSTRUCTOR = ['id', 'name', 'dept_name', 'salary']

    ## data
    LIST_INSTRUCTOR_INFO = [
        ('10101', 'Srinivasan', 'Comp. Sci.', '65000'),
        ('12121', 'Wu', 'Finance', '90000'),
        ('15151', 'Mozart', 'Music', '40000'),
        ('22222', 'Einstein', 'Physics', '95000'),
        ('32343', 'El Said', 'History', '60000'),
        ('33456', 'Gold', 'Physics', '87000'),
        ('45565', 'Katz', 'Comp. Sci.', '75000'),
        ('58583', 'Califieri', 'History', '62000'),
        ('76543', 'Singh', 'Finance', '80000'),
        ('76766', 'Crick', 'Biology', '72000'),
        ('83821', 'Brandt', 'Comp. Sci.', '92000'),
        ('98345', 'Kim', 'Elec. Eng.', '80000'),
    ]

    ## prepared statement
    query = sql.SQL(SQL_TEMPLATE_INSERT).format(
        sql.Identifier(SCNAME, 'instructor'),
        sql.SQL(', ').join(map(sql.Identifier, COLNAMES_INSTRUCTOR)),
        sql.SQL(', ').join(sql.Placeholder() * len(COLNAMES_INSTRUCTOR))
    )
    
    with conn.pipeline() as p:
        for data in LIST_INSTRUCTOR_INFO:
            conn.execute(query, data)

    conn.commit()    

In [14]:
with psycopg.connect(CONNECTION) as conn:
    
    cur = conn.execute(sql.SQL("SELECT * FROM {}").format(sql.Identifier(SCNAME, 'instructor')))

    records = cur.fetchall()
    for r in records:
        print(r)

('10101', 'Srinivasan', 'Comp. Sci.', Decimal('65000.00'))
('12121', 'Wu', 'Finance', Decimal('90000.00'))
('15151', 'Mozart', 'Music', Decimal('40000.00'))
('22222', 'Einstein', 'Physics', Decimal('95000.00'))
('32343', 'El Said', 'History', Decimal('60000.00'))
('33456', 'Gold', 'Physics', Decimal('87000.00'))
('45565', 'Katz', 'Comp. Sci.', Decimal('75000.00'))
('58583', 'Califieri', 'History', Decimal('62000.00'))
('76543', 'Singh', 'Finance', Decimal('80000.00'))
('76766', 'Crick', 'Biology', Decimal('72000.00'))
('83821', 'Brandt', 'Comp. Sci.', Decimal('92000.00'))
('98345', 'Kim', 'Elec. Eng.', Decimal('80000.00'))


# Cartesian Product

In [15]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("""
        SELECT DISTINCT T.{name}, T.{salary}
        FROM {table_name} AS T, {table_name} AS S
        WHERE T.{salary} > S.{salary} AND S.{dept_name} = %s
        ORDER BY T.{salary}
    """).format(
        table_name = sql.Identifier(SCNAME, 'instructor'),
        name       = sql.Identifier('name'),
        salary     = sql.Identifier('salary'),
        dept_name  = sql.Identifier('dept_name')
    )

    cur = conn.execute(query, ('Comp. Sci.', ))

    for r in cur.fetchall():
        print(r)

('Crick', Decimal('72000.00'))
('Katz', Decimal('75000.00'))
('Singh', Decimal('80000.00'))
('Kim', Decimal('80000.00'))
('Gold', Decimal('87000.00'))
('Wu', Decimal('90000.00'))
('Brandt', Decimal('92000.00'))
('Einstein', Decimal('95000.00'))


# String Operations

In [16]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("""
        SELECT {name}
        FROM {instructor}
        WHERE {name} LIKE %s
    """).format(
        name = sql.Identifier('name'),
        instructor = sql.Identifier(SCNAME, 'instructor'),
    )

    for r in conn.execute(query, ('%%a%%', )).fetchall():
        print(r)

('Srinivasan',)
('Mozart',)
('El Said',)
('Katz',)
('Califieri',)
('Brandt',)


# Where Clause Predicates

In [17]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("""
        SELECT {name}
        FROM {instructor}
        WHERE {salary} BETWEEN %s AND %s
    """).format(
        name = sql.Identifier('name'),
        instructor = sql.Identifier(SCNAME, 'instructor'),
        salary = sql.Identifier('salary'),
    )

    for r in conn.execute(query, ('90000', 100000)):
        print(r)    

('Wu',)
('Einstein',)
('Brandt',)


# Set Operations

In [18]:
with psycopg.connect(CONNECTION) as conn:

    subquery1 = sql.SQL("""
        SELECT {course_id}, {title}
        FROM {course}
        WHERE {dept_name} = %s
    """).format(
        course_id = sql.Identifier('course_id'),
        title = sql.Identifier('title'),
        course = sql.Identifier(SCNAME, 'course'),
        dept_name = sql.Identifier('dept_name')
    )

    subquery2 = sql.SQL("""
        SELECT {}
        FROM {}
        WHERE {} = %s
    """).format(
        sql.SQL(', ').join([sql.Identifier('course_id'), sql.Identifier('title')]),
        sql.Identifier(SCNAME, 'course'),
        sql.Identifier('dept_name')
    )

    query = sql.SQL('({}) UNION ({})').format(subquery1, subquery2)
    query = sql.SQL('{} ORDER BY {}').format(query, sql.Identifier('course_id'))
    
    for r in conn.execute(query, ('Comp. Sci.', 'Biology')).fetchall():
        print(r)

('BIO-101', 'Intro. to Biology')
('BIO-301', 'Genetics')
('BIO-399', 'Computational Biology')
('CS-101', 'Intro. to Computer Science')
('CS-190', 'Game Design')
('CS-315', 'Robotics')
('CS-319', 'Image Processing')
('CS-347', 'Database System Concepts')


# Aggregate Functions

In [19]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("""
        SELECT {dept_name}, AVG({salary}) AS {avg_salary}
        FROM {instructor}
        GROUP BY {dept_name}
    """).format(
        dept_name = sql.Identifier('dept_name'),
        salary = sql.Identifier('salary'),
        avg_salary = sql.Identifier('avg_salary'),
        instructor = sql.Identifier(SCNAME, 'instructor'),
    )
    
    for r in conn.execute(query).fetchall():
        print(r)

('Finance', Decimal('85000.000000000000'))
('History', Decimal('61000.000000000000'))
('Physics', Decimal('91000.000000000000'))
('Music', Decimal('40000.000000000000'))
('Comp. Sci.', Decimal('77333.333333333333'))
('Biology', Decimal('72000.000000000000'))
('Elec. Eng.', Decimal('80000.000000000000'))


# Nested subqueries

In [20]:
with psycopg.connect(CONNECTION) as conn:

    query_sub = sql.SQL("""
        SELECT {salary}
        FROM {instructor}
        WHERE {dept_name} = %s
    """).format(
        salary = sql.Identifier('salary'),
        instructor = sql.Identifier(SCNAME, 'instructor'),
        dept_name = sql.Identifier('dept_name'),
    )

    query_main = sql.SQL("""
        SELECT {name}, {salary}
        FROM {instructor}
        WHERE {salary} > SOME({query_sub})
        ORDER BY {salary}
    """).format(
        name = sql.Identifier('name'),
        salary = sql.Identifier('salary'),
        instructor = sql.Identifier(SCNAME, 'instructor'),
        query_sub = query_sub
    )

    for r in conn.execute(query_main, ('Comp. Sci.', )):
        print(r)

('Crick', Decimal('72000.00'))
('Katz', Decimal('75000.00'))
('Kim', Decimal('80000.00'))
('Singh', Decimal('80000.00'))
('Gold', Decimal('87000.00'))
('Wu', Decimal('90000.00'))
('Brandt', Decimal('92000.00'))
('Einstein', Decimal('95000.00'))


# Delete records

In [21]:
with psycopg.connect(CONNECTION) as conn:

    query_sub = sql.SQL("""
        SELECT AVG({salary})
        FROM {instructor}
    """).format(
        salary = sql.Identifier('salary'),
        instructor = sql.Identifier(SCNAME, 'instructor')
    )

    query_main = sql.SQL("""
        DELETE
        FROM {instructor}
        WHERE {salary} > ({query_sub})
    """).format(
        instructor = sql.Identifier(SCNAME, 'instructor'),
        salary = sql.Identifier('salary'),
        query_sub = query_sub
    )

    conn.execute(query_main)

    conn.commit()

In [22]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(SCNAME, 'instructor'))
    
    for r in conn.execute(query).fetchall():
        print(r)

('10101', 'Srinivasan', 'Comp. Sci.', Decimal('65000.00'))
('15151', 'Mozart', 'Music', Decimal('40000.00'))
('32343', 'El Said', 'History', Decimal('60000.00'))
('58583', 'Califieri', 'History', Decimal('62000.00'))
('76766', 'Crick', 'Biology', Decimal('72000.00'))


# Update records

In [23]:
with psycopg.connect(CONNECTION) as conn:

    query_sub = sql.SQL("""
        WHEN {0} <= 100000
        THEN {0} * 1.05
        ELSE {0} * 1.03
    """).format(sql.Identifier('salary'))

    query_main = sql.SQL("""
        UPDATE {instructor}
        SET {salary} = CASE {query_sub} END
    """).format(
        instructor = sql.Identifier(SCNAME, 'instructor'),
        salary = sql.Identifier('salary'),
        query_sub = query_sub,
    )

    conn.execute(query_main)

    conn.commit()

In [24]:
with psycopg.connect(CONNECTION) as conn:

    query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(SCNAME, 'instructor'))
    
    for r in conn.execute(query).fetchall():
        print(r)

('10101', 'Srinivasan', 'Comp. Sci.', Decimal('68250.00'))
('15151', 'Mozart', 'Music', Decimal('42000.00'))
('32343', 'El Said', 'History', Decimal('63000.00'))
('58583', 'Califieri', 'History', Decimal('65100.00'))
('76766', 'Crick', 'Biology', Decimal('75600.00'))
